<a href="https://colab.research.google.com/github/peterbabulik/QuantumWalker/blob/main/3WalkersR30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cirq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 597.5/597.5 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.1/425.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 70.3 MB/s eta 0:00:00


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import time
import os

# --- Parameters ---
N_SITES_1D = 9
DEPTH = 7
# Adjust initial positions for 3 walkers to be distinct
INITIAL_POS_A = N_SITES_1D // 2 - N_SITES_1D // 3
INITIAL_POS_B = N_SITES_1D // 2
INITIAL_POS_C = N_SITES_1D // 2 + N_SITES_1D // 3

INITIAL_COIN_A = 0
INITIAL_COIN_B = 1
INITIAL_COIN_C = 0 # Example for Walker C

INITIAL_CA_CENTER_ONE = True
CA_RULE_NUMBER = 30 # Specifically Rule 30
P_THRESHOLD_FEEDBACK = 0.1
FEEDBACK_TYPE = "flip"

# --- CA Rule Update Function (Generalized) ---
def update_ca_line(current_line, rule_number):
    width = len(current_line)
    new_line = np.zeros(width, dtype=int)
    for i in range(width):
        left_neighbor = current_line[(i - 1 + width) % width]
        center_cell = current_line[i]
        right_neighbor = current_line[(i + 1) % width]
        pattern_val = (left_neighbor << 2) | (center_cell << 1) | right_neighbor
        if (rule_number >> pattern_val) & 1:
            new_line[i] = 1
        else:
            new_line[i] = 0
    return new_line

# --- Coin Matrices ---
H_1Q = (1/np.sqrt(2))*np.array([[1,1],[1,-1]],dtype=np.complex128)
X_1Q = np.array([[0,1],[1,0]],dtype=np.complex128)
I_1Q = np.eye(2,dtype=np.complex128)
COIN0_OP_DEFAULT = H_1Q
COIN1_OP_DEFAULT = X_1Q @ H_1Q

# --- Helper Functions (1D QW - Single Walker Indexing) ---
def get_1d_index(coin_val, site_pos, n_sites_1d):
    if not (0 <= site_pos < n_sites_1d and 0 <= coin_val <= 1):
        raise IndexError(f"Invalid coin ({coin_val}) or pos ({site_pos}) for 1D index (N_sites={n_sites_1d})")
    return coin_val + 2 * site_pos

def get_1d_coin_pos_from_index(k, n_sites_1d):
    state_dim_1d = 2 * n_sites_1d
    if not (0 <= k < state_dim_1d):
        raise IndexError(f"Invalid k ({k}) for 1D coin/pos (StateDim={state_dim_1d})")
    coin_val = k % 2
    site_pos = k // 2
    return coin_val, site_pos

# --- Helper Functions (3 Walkers on 1D Grid) ---
def get_3walker_index(cA, pA, cB, pB, cC, pC, n_sites_1d):
    d_1w = 2 * n_sites_1d
    idx_A = get_1d_index(cA, pA, n_sites_1d)
    idx_B = get_1d_index(cB, pB, n_sites_1d)
    idx_C = get_1d_index(cC, pC, n_sites_1d)
    if not (0 <= idx_A < d_1w and 0 <= idx_B < d_1w and 0 <= idx_C < d_1w):
        raise IndexError(f"Internal 1D index out of bounds during 3-walker indexing.")
    return idx_A + d_1w * idx_B + d_1w * d_1w * idx_C

def get_3walker_coords_from_index(k, n_sites_1d):
    d_1w = 2 * n_sites_1d
    d_1w_sq = d_1w * d_1w
    d_tot = d_1w_sq * d_1w
    if not (0 <= k < d_tot):
        raise IndexError(f"Invalid k ({k}) for 3-walker (Dim={d_tot})")

    idx_C = k // d_1w_sq
    remainder_after_C = k % d_1w_sq
    idx_B = remainder_after_C // d_1w
    idx_A = remainder_after_C % d_1w

    cA, pA = get_1d_coin_pos_from_index(idx_A, n_sites_1d)
    cB, pB = get_1d_coin_pos_from_index(idx_B, n_sites_1d)
    cC, pC = get_1d_coin_pos_from_index(idx_C, n_sites_1d)
    return cA, pA, cB, pB, cC, pC

# --- Initial State Preparation (3 Walkers) ---
def prepare_initial_state_3walkers(n_sites_1d, pA_init, cA_init, pB_init, cB_init, pC_init, cC_init):
    d_1w = 2 * n_sites_1d
    state_dim_3w = d_1w * d_1w * d_1w
    initial_state_vector = np.zeros(state_dim_3w, dtype=np.complex128)
    try:
        start_idx = get_3walker_index(cA_init, pA_init, cB_init, pB_init, cC_init, pC_init, n_sites_1d)
        initial_state_vector[start_idx] = 1.0
        print(f"Initialized Walker A at ({pA_init}, c{cA_init}), B at ({pB_init}, c{cB_init}), C at ({pC_init}, c{cC_init}). Index: {start_idx}")
    except IndexError as e:
        print(f"Error during 3-walker initial state prep: {e}")
    return initial_state_vector

# --- Build Single Walker Operators (for kronecker product basis) ---
def build_single_qw_coin_matrix_for_site(current_ca_pattern_site_val, coin0_op, coin1_op):
    return coin0_op if current_ca_pattern_site_val == 0 else coin1_op

def build_single_qw_shift_matrix(n_sites_1d):
    d_1w = 2 * n_sites_1d
    S_1w = np.zeros((d_1w, d_1w), dtype=np.complex128)
    for k_in_1w in range(d_1w):
        coin_val, site_pos = get_1d_coin_pos_from_index(k_in_1w, n_sites_1d)
        new_pos = (site_pos - 1 + n_sites_1d) % n_sites_1d if coin_val == 0 else (site_pos + 1) % n_sites_1d
        k_out_1w = get_1d_index(coin_val, new_pos, n_sites_1d)
        S_1w[k_out_1w, k_in_1w] = 1.0
    return S_1w

# --- Build Full QW Step Operator for 3 Walkers ---
def build_3walker_full_step_operator(n_sites_1d, current_ca_pattern, coin0_op, coin1_op, S_A_g, S_B_g, S_C_g):
    d_1w = 2 * n_sites_1d
    state_dim_3w = d_1w * d_1w * d_1w

    U_Coin_A = np.zeros((state_dim_3w, state_dim_3w), dtype=np.complex128)
    U_Coin_B = np.zeros((state_dim_3w, state_dim_3w), dtype=np.complex128)
    U_Coin_C = np.zeros((state_dim_3w, state_dim_3w), dtype=np.complex128)

    for k_in_joint in range(state_dim_3w):
        cA_in, pA_in, cB_in, pB_in, cC_in, pC_in = get_3walker_coords_from_index(k_in_joint, n_sites_1d)

        # Coin A
        coin_matrix_for_A = build_single_qw_coin_matrix_for_site(current_ca_pattern[pA_in], coin0_op, coin1_op)
        for cA_out in range(2):
            amplitude_A = coin_matrix_for_A[cA_out, cA_in]
            if np.abs(amplitude_A) > 1e-9:
                k_out_A = get_3walker_index(cA_out, pA_in, cB_in, pB_in, cC_in, pC_in, n_sites_1d)
                U_Coin_A[k_out_A, k_in_joint] = amplitude_A
        # Coin B
        coin_matrix_for_B = build_single_qw_coin_matrix_for_site(current_ca_pattern[pB_in], coin0_op, coin1_op)
        for cB_out in range(2):
            amplitude_B = coin_matrix_for_B[cB_out, cB_in]
            if np.abs(amplitude_B) > 1e-9:
                k_out_B = get_3walker_index(cA_in, pA_in, cB_out, pB_in, cC_in, pC_in, n_sites_1d)
                U_Coin_B[k_out_B, k_in_joint] = amplitude_B
        # Coin C
        coin_matrix_for_C = build_single_qw_coin_matrix_for_site(current_ca_pattern[pC_in], coin0_op, coin1_op)
        for cC_out in range(2):
            amplitude_C = coin_matrix_for_C[cC_out, cC_in]
            if np.abs(amplitude_C) > 1e-9:
                k_out_C = get_3walker_index(cA_in, pA_in, cB_in, pB_in, cC_out, pC_in, n_sites_1d)
                U_Coin_C[k_out_C, k_in_joint] = amplitude_C

    # Apply coins first, then shifts (consistent with 2-walker)
    U_step = S_C_g @ S_B_g @ S_A_g @ U_Coin_C @ U_Coin_B @ U_Coin_A
    return U_step

# --- Observables (3 Walkers) ---
def calculate_p_pos_3walkers(joint_sv, n_sites_1d, walker_id='A'):
    d_1w = 2 * n_sites_1d; d_tot = d_1w**3
    probs_1d = np.zeros(n_sites_1d)
    if len(joint_sv) != d_tot: return probs_1d

    for k_joint in range(d_tot):
        cA, pA, cB, pB, cC, pC = get_3walker_coords_from_index(k_joint, n_sites_1d)
        amp_sq = np.abs(joint_sv[k_joint])**2
        if walker_id == 'A': probs_1d[pA] += amp_sq
        elif walker_id == 'B': probs_1d[pB] += amp_sq
        elif walker_id == 'C': probs_1d[pC] += amp_sq
    return probs_1d

def calculate_coin_entanglement_3walkers(joint_sv, n_sites_1d, walker_id='A'):
    d_1w = 2 * n_sites_1d; d_tot = d_1w**3; coin_dim = 2
    if len(joint_sv) != d_tot: return np.nan

    norm = np.linalg.norm(joint_sv)
    if norm < 1e-9: return 0.0
    current_psi = joint_sv / norm if np.abs(norm - 1.0) > 1e-6 else joint_sv

    rho_full = np.outer(current_psi, np.conjugate(current_psi))
    rho_coin_walker = np.zeros((coin_dim, coin_dim), dtype=np.complex128)

    for k_joint_row in range(d_tot):
        cA_r, pA_r, cB_r, pB_r, cC_r, pC_r = get_3walker_coords_from_index(k_joint_row, n_sites_1d)
        for k_joint_col in range(d_tot):
            cA_c, pA_c, cB_c, pB_c, cC_c, pC_c = get_3walker_coords_from_index(k_joint_col, n_sites_1d)

            if walker_id == 'A':
                if pA_r == pA_c and \
                   cB_r == cB_c and pB_r == pB_c and \
                   cC_r == cC_c and pC_r == pC_c:
                    rho_coin_walker[cA_r, cA_c] += rho_full[k_joint_row, k_joint_col]
            elif walker_id == 'B':
                if pB_r == pB_c and \
                   cA_r == cA_c and pA_r == pA_c and \
                   cC_r == cC_c and pC_r == pC_c:
                    rho_coin_walker[cB_r, cB_c] += rho_full[k_joint_row, k_joint_col]
            elif walker_id == 'C':
                if pC_r == pC_c and \
                   cA_r == cA_c and pA_r == pA_c and \
                   cB_r == cB_c and pB_r == pB_c:
                     rho_coin_walker[cC_r, cC_c] += rho_full[k_joint_row, k_joint_col]

    tr_rho_coin = np.trace(rho_coin_walker)
    if abs(tr_rho_coin) < 1e-9: return 0.0
    if np.abs(tr_rho_coin - 1.0) > 1e-6 : rho_coin_walker /= tr_rho_coin

    eigs = np.linalg.eigvalsh(rho_coin_walker); ent = 0.0; epsilon = 1e-12
    for e_val in eigs: ent -= (e_val * np.log2(e_val) if e_val > epsilon else 0.0)
    return max(0.0, np.real(ent))

# --- Simulation Loop (3 Walkers with Two-Way Coupled Dynamic CA Coin) ---
def run_1d_qw_3walkers_two_way_coupling(
    n_sites_1d, depth,
    pA_init, cA_init, pB_init, cB_init, pC_init, cC_init,
    initial_ca_center_one, ca_rule_number,
    coin0_op, coin1_op,
    p_threshold, feedback_type
):
    current_qw_state = prepare_initial_state_3walkers(n_sites_1d, pA_init, cA_init, pB_init, cB_init, pC_init, cC_init)
    current_ca_line = np.zeros(n_sites_1d, dtype=int)
    if initial_ca_center_one and n_sites_1d > 0:
        center_idx = n_sites_1d // 2
        current_ca_line[center_idx if n_sites_1d % 2 == 1 else center_idx -1] = 1

    prob_A_hist = np.full((depth + 1, n_sites_1d), np.nan)
    prob_B_hist = np.full((depth + 1, n_sites_1d), np.nan)
    prob_C_hist = np.full((depth + 1, n_sites_1d), np.nan)
    ent_A_hist = np.full(depth + 1, np.nan)
    ent_B_hist = np.full(depth + 1, np.nan)
    ent_C_hist = np.full(depth + 1, np.nan)
    ca_hist_3w = [current_ca_line.copy()]

    prob_A_hist[0, :] = calculate_p_pos_3walkers(current_qw_state, n_sites_1d, 'A')
    prob_B_hist[0, :] = calculate_p_pos_3walkers(current_qw_state, n_sites_1d, 'B')
    prob_C_hist[0, :] = calculate_p_pos_3walkers(current_qw_state, n_sites_1d, 'C')
    ent_A_hist[0] = calculate_coin_entanglement_3walkers(current_qw_state, n_sites_1d, 'A')
    ent_B_hist[0] = calculate_coin_entanglement_3walkers(current_qw_state, n_sites_1d, 'B')
    ent_C_hist[0] = calculate_coin_entanglement_3walkers(current_qw_state, n_sites_1d, 'C')

    print(f"\nStarting 3-Walker 1D QW (CA Rule {ca_rule_number}) for {depth} steps...")
    start_time = time.time()

    d_1w = 2 * n_sites_1d
    I_1w = np.eye(d_1w, dtype=np.complex128)
    S_1w = build_single_qw_shift_matrix(n_sites_1d)
    S_A_global_const = np.kron(S_1w, np.kron(I_1w, I_1w))
    S_B_global_const = np.kron(I_1w, np.kron(S_1w, I_1w))
    S_C_global_const = np.kron(I_1w, np.kron(I_1w, S_1w))

    for step in range(depth):
        U_step = build_3walker_full_step_operator(n_sites_1d, current_ca_line, coin0_op, coin1_op,
                                                  S_A_global_const, S_B_global_const, S_C_global_const)
        current_qw_state = U_step @ current_qw_state
        norm_qw = np.linalg.norm(current_qw_state)
        if abs(norm_qw) > 1e-9 : current_qw_state /= norm_qw
        else: print(f"Warning: QW Norm zero at step {step+1}."); break

        pA_current = calculate_p_pos_3walkers(current_qw_state, n_sites_1d, 'A')
        pB_current = calculate_p_pos_3walkers(current_qw_state, n_sites_1d, 'B')
        pC_current = calculate_p_pos_3walkers(current_qw_state, n_sites_1d, 'C')
        prob_A_hist[step + 1, :] = pA_current
        prob_B_hist[step + 1, :] = pB_current
        prob_C_hist[step + 1, :] = pC_current
        ent_A_hist[step+1] = calculate_coin_entanglement_3walkers(current_qw_state, n_sites_1d, 'A')
        ent_B_hist[step+1] = calculate_coin_entanglement_3walkers(current_qw_state, n_sites_1d, 'B')
        ent_C_hist[step+1] = calculate_coin_entanglement_3walkers(current_qw_state, n_sites_1d, 'C')

        ca_line_for_next_update = current_ca_line.copy()
        for i in range(n_sites_1d):
            if pA_current[i] > p_threshold or pB_current[i] > p_threshold or pC_current[i] > p_threshold:
                if feedback_type=="flip": ca_line_for_next_update[i]=1-ca_line_for_next_update[i]
                elif feedback_type=="set_to_one": ca_line_for_next_update[i]=1
                elif feedback_type=="set_to_zero": ca_line_for_next_update[i]=0
        current_ca_line = update_ca_line(ca_line_for_next_update, ca_rule_number)
        ca_hist_3w.append(current_ca_line.copy())

        if (step + 1) % 10 == 0 or step == depth -1 :
            print(f"  3W-QW Step {step + 1}/{depth} completed. "
                  f"S(A)={f'{ent_A_hist[step+1]:.3f}' if np.isfinite(ent_A_hist[step+1]) else 'NaN'}, "
                  f"S(B)={f'{ent_B_hist[step+1]:.3f}' if np.isfinite(ent_B_hist[step+1]) else 'NaN'}, "
                  f"S(C)={f'{ent_C_hist[step+1]:.3f}' if np.isfinite(ent_C_hist[step+1]) else 'NaN'}")

    end_time = time.time()
    print(f"3-Walker QW Evolution complete. Time: {end_time - start_time:.2f} seconds.")

    return {
        "prob_A_hist": prob_A_hist, "prob_B_hist": prob_B_hist, "prob_C_hist": prob_C_hist,
        "ent_A_hist": ent_A_hist, "ent_B_hist": ent_B_hist, "ent_C_hist": ent_C_hist,
        "ca_history": np.array(ca_hist_3w),
        "params": { "n_sites":n_sites_1d,"depth":depth, "ca_rule":ca_rule_number,
                    "pA_init":pA_init, "cA_init":cA_init,
                    "pB_init":pB_init, "cB_init":cB_init,
                    "pC_init":pC_init, "cC_init":cC_init,
                    "p_thresh":p_threshold, "feedback":feedback_type}
    }

# --- Plotting and Summary for 3 Walkers ---
def plot_3walker_results(results, save_path=None):
    params = results["params"]
    pA, pB, pC = results["prob_A_hist"], results["prob_B_hist"], results["prob_C_hist"]
    eA, eB, eC = results["ent_A_hist"], results["ent_B_hist"], results["ent_C_hist"]
    ca_h = results["ca_history"]
    n_sites = params["n_sites"]; depth = params["depth"]

    fig, axs = plt.subplots(5, 1, figsize=(12, 22), gridspec_kw={'height_ratios': [1.5, 3, 3, 3, 2]})
    title_str=f"3-Walker QW & CA Rule {params['ca_rule']} (N={n_sites}, D={depth}, Pth={params['p_thresh']:.2f}, Fb='{params['feedback']}')"
    fig.suptitle(title_str, fontsize=14)

    axs[0].imshow(ca_h,cmap='binary',aspect='auto',interpolation='nearest'); axs[0].set_title("CA Evolution"); axs[0].set_xlabel("Site"); axs[0].set_yticks([])

    time_extent = [0, depth, 0, n_sites-1]
    def plot_prob_dist(ax, prob_hist_walker, walker_label, cmap_choice):
        prob_st=prob_hist_walker.T;pos_p=prob_st[prob_st>1e-9];min_v=pos_p.min() if len(pos_p)>0 else 1e-9;max_v=prob_st.max()
        norm_v=colors.LogNorm(vmin=min_v,vmax=max(max_v,min_v+1e-6) if max_v > min_v else min_v + 1e-6 )
        im=ax.imshow(prob_st,aspect='auto',origin='lower',cmap=cmap_choice,norm=norm_v,extent=time_extent)
        plt.colorbar(im,ax=ax,label=f"P_{walker_label}(x,t) (log)");ax.set_title(f"Walker {walker_label} P(x,t)");ax.set_xlabel("Time Step");ax.set_ylabel("Site (x)")

    plot_prob_dist(axs[1], pA, 'A', 'viridis')
    plot_prob_dist(axs[2], pB, 'B', 'magma')
    plot_prob_dist(axs[3], pC, 'C', 'plasma') # Different cmap for C

    ts=np.arange(depth+1)
    axs[4].plot(ts,eA,marker='.',ls='-',color='blue',label='S(A)'); axs[4].plot(ts,eB,marker='.',ls='-',color='red',label='S(B)')
    axs[4].plot(ts,eC,marker='.',ls='-',color='green',label='S(C)')
    axs[4].set_title("Coin Entanglement S(t)"); axs[4].set_xlabel("Time Step"); axs[4].set_ylabel("Entropy S"); axs[4].grid(True); axs[4].legend();axs[4].set_ylim(bottom=-0.05,top=1.05)

    plt.tight_layout(rect=[0,0,1,0.96])

    if save_path:
        plt.savefig(save_path)
        print(f"Plot saved to {save_path}")
        plt.close(fig)
    else:
        plt.show()

def generate_3walker_text_output(results):
    params = results["params"]
    p_hists = {"A": results["prob_A_hist"], "B": results["prob_B_hist"], "C": results["prob_C_hist"]}
    e_hists = {"A": results["ent_A_hist"], "B": results["ent_B_hist"], "C": results["ent_C_hist"]}
    p_inits = {"A": params["pA_init"], "B": params["pB_init"], "C": params["pC_init"]}
    c_inits = {"A": params["cA_init"], "B": params["cB_init"], "C": params["cC_init"]}

    output_lines = [f"\n--- 3-Walker QW (CA Rule {params['ca_rule']}) Summary ---"]
    output_lines.append(f"N_sites={params['n_sites']}, Depth={params['depth']}, Pth={params['p_thresh']:.2f}, Fb='{params['feedback']}'")
    for walker_label in ["A", "B", "C"]:
        output_lines.append(f"  Walker {walker_label}: StartPos={p_inits[walker_label]}, StartCoin={c_inits[walker_label]}")

    for walker_label in ["A", "B", "C"]:
        p_hist = p_hists[walker_label]
        e_hist = e_hists[walker_label]
        output_lines.append(f"\nWalker {walker_label} - Top 5 Final Probabilities (Site: Prob):")
        if p_hist.ndim == 2 and p_hist.shape[0] > 0:
            final_probs = p_hist[-1, :]; sorted_indices = np.argsort(final_probs)[::-1]
            count = 0
            for i_idx in range(min(5, params['n_sites'])):
                idx = sorted_indices[i_idx]
                if np.isfinite(final_probs[idx]) and final_probs[idx] > 1e-9:
                    output_lines.append(f"  Site {idx}: {final_probs[idx]:.5f}"); count+=1
            if count == 0: output_lines.append("  (No significant probabilities at final step or probabilities are NaN)")
        else: output_lines.append("  (Probability history not available)")

        output_lines.append(f"Walker {walker_label} - Coin Entanglement vs. Time:")
        num_steps = len(e_hist)
        indices_show = [0] + ([num_steps // 2] if num_steps > 2 else []) + ([num_steps - 1] if num_steps > 1 else [])
        for idx_show in sorted(list(set(indices_show))):
            if 0 <= idx_show < len(e_hist):
                 s_str = f"{e_hist[idx_show]:.6f}" if np.isfinite(e_hist[idx_show]) else "nan"; output_lines.append(f"  Step {idx_show}: S = {s_str}")
            else: output_lines.append(f"  Step {idx_show}: S = (Index out of bounds)")
    output_lines.append("--- End Summary ---")
    return "\n".join(output_lines)

# --- Main Execution ---
if __name__ == "__main__":
    # Only Rule 30 for 3 walkers
    current_rule_number = CA_RULE_NUMBER

    output_dir = "qw_ca_3walker_rule30"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created directory: {output_dir}")

    print(f"\n===== Running simulation for 3 Walkers with CA Rule: {current_rule_number} =====")

    results_3w = run_1d_qw_3walkers_two_way_coupling(
        n_sites_1d=N_SITES_1D, depth=DEPTH,
        pA_init=INITIAL_POS_A, cA_init=INITIAL_COIN_A,
        pB_init=INITIAL_POS_B, cB_init=INITIAL_COIN_B,
        pC_init=INITIAL_POS_C, cC_init=INITIAL_COIN_C,
        initial_ca_center_one=INITIAL_CA_CENTER_ONE,
        ca_rule_number=current_rule_number,
        coin0_op=COIN0_OP_DEFAULT, coin1_op=COIN1_OP_DEFAULT,
        p_threshold=P_THRESHOLD_FEEDBACK, feedback_type=FEEDBACK_TYPE
    )

    text_summary_3w = generate_3walker_text_output(results_3w)
    print(text_summary_3w)

    plot_filename = os.path.join(output_dir, f"rule{current_rule_number:03d}_3W_N{N_SITES_1D}_D{DEPTH}.png")
    plot_3walker_results(results_3w, save_path=plot_filename)

    print(f"\n<<<<< 3-WALKER SIMULATION FOR RULE {current_rule_number} COMPLETE >>>>>")
    print(f"Plot saved in ./{output_dir}/")

Created directory: qw_ca_3walker_rule30

===== Running simulation for 3 Walkers with CA Rule: 30 =====
Initialized Walker A at (1, c0), B at (4, c1), C at (7, c0). Index: 4700

Starting 3-Walker 1D QW (CA Rule 30) for 7 steps...
  3W-QW Step 7/7 completed. S(A)=0.780, S(B)=0.783, S(C)=0.755
3-Walker QW Evolution complete. Time: 2293.86 seconds.

--- 3-Walker QW (CA Rule 30) Summary ---
N_sites=9, Depth=7, Pth=0.10, Fb='flip'
  Walker A: StartPos=1, StartCoin=0
  Walker B: StartPos=4, StartCoin=1
  Walker C: StartPos=7, StartCoin=0

Walker A - Top 5 Final Probabilities (Site: Prob):
  Site 0: 0.47656
  Site 2: 0.22656
  Site 5: 0.13281
  Site 7: 0.10156
  Site 4: 0.03906
Walker A - Coin Entanglement vs. Time:
  Step 0: S = 0.000000
  Step 4: S = 0.696212
  Step 7: S = 0.780058

Walker B - Top 5 Final Probabilities (Site: Prob):
  Site 7: 0.32031
  Site 0: 0.28906
  Site 1: 0.19531
  Site 3: 0.10156
  Site 8: 0.03906
Walker B - Coin Entanglement vs. Time:
  Step 0: S = 0.000000
  Step 4: